<a href="https://colab.research.google.com/github/GabrielDeml/CatvsDogClassifier/blob/master/CatVsDog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras import datasets, layers, models

import IPython.display as display
import pathlib
import tensorflow as tf
import os
tf.enable_eager_execution()
tf.__version__

Using TensorFlow backend.


'1.14.0-rc1'

In [0]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

In [0]:
!mkdir .kaggle

mkdir: cannot create directory ‘.kaggle’: File exists


In [0]:
import json
token = {"username":"username","key":"KEY"}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!chmod 600 /content/.kaggle/kaggle.json

In [0]:
mkdir ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [0]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [0]:
!kaggle competitions download -c dogs-vs-cats -p /content

sampleSubmission.csv: Skipping, found more recently modified local copy (use --force to force download)
test1.zip: Skipping, found more recently modified local copy (use --force to force download)
train.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
print ("train: {}, test1 {}".format(os.path.exists("train"), os.path.exists("test1")))
if not (os.path.exists("train") and os.path.exists("test1")):
  !unzip -q train.zip
  !unzip -q test1.zip
else:
  print("We already did this")

train: True, test1 True
We already did this


In [0]:
!mkdir train/cat
!mkdir train/dog
!mv train/cat* train/cat
!mv train/dog* train/dog

mkdir: cannot create directory ‘train/cat’: File exists
mkdir: cannot create directory ‘train/dog’: File exists
mv: cannot move 'train/cat' to a subdirectory of itself, 'train/cat/cat'
mv: cannot move 'train/dog' to a subdirectory of itself, 'train/dog/dog'


In [0]:
data_root = pathlib.Path("train")

In [0]:
for item in data_root.iterdir():
  print(item)

train/cat
train/dog


In [0]:
import random
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
image_count

25000

In [0]:
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_names

['cat', 'dog']

In [0]:
label_to_index = dict((name, index) for index,name in enumerate(label_names))
label_to_index

{'cat': 0, 'dog': 1}

In [0]:
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]

print("First 10 labels indices: ", all_image_labels[:10])

First 10 labels indices:  [0, 1, 0, 1, 0, 1, 0, 0, 1, 0]


In [0]:
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [192, 192])
  image /= 255.0  # normalize to [0,1] range

  return image

In [0]:
def load_and_preprocess_image(path):
  image = tf.read_file(path)
  return preprocess_image(image)

In [0]:
ds = tf.data.Dataset.from_tensor_slices((all_image_paths, all_image_labels))

# The tuples are unpacked into the positional arguments of the mapped function
def load_and_preprocess_from_path_label(path, label):
  return load_and_preprocess_image(path), label

image_label_ds = ds.map(load_and_preprocess_from_path_label)
image_label_ds

<DatasetV1Adapter shapes: ((192, 192, 3), ()), types: (tf.float32, tf.int32)>

In [0]:
ds = image_label_ds.apply(
tf.data.experimental.shuffle_and_repeat(buffer_size=10000))
ds = ds.batch(64)
ds = ds.prefetch(buffer_size=1000)
ds = ds.repeat()
ds
# ds = image_label_ds.apply(
#   tf.data.experimental.shuffle_and_repeat(buffer_size=image_count))
# ds = ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
# ds
# ds = ds.shuffle(buffer_size=10000) ## 10000: size of sample/record pool for random selection
# ds = ds.batch(32) ## 32: number of samples/records per batch (to be read into memory)
# ds = ds.repeat() ## None: keep repeating

W0618 02:54:20.973101 139729980331904 deprecation.py:323] From <ipython-input-21-29cd976dcf6a>:2: shuffle_and_repeat (from tensorflow.python.data.experimental.ops.shuffle_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.


<DatasetV1Adapter shapes: ((?, 192, 192, 3), (?,)), types: (tf.float32, tf.int32)>

In [0]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(192, 192, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 190, 190, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 95, 95, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 93, 93, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 46, 46, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 123904)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                7

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=["accuracy"])

In [0]:
model.fit(ds, epochs=600, steps_per_epoch=3)

Epoch 1/600


W0618 02:54:47.236327 139729980331904 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


3/3 [==============================] - 30s 10s/step - loss: 2.5614 - acc: 0.5208
Epoch 2/600
3/3 [==============================] - 0s 75ms/step - loss: 0.7420 - acc: 0.5729
Epoch 3/600
3/3 [==============================] - 0s 75ms/step - loss: 0.6887 - acc: 0.5365
Epoch 4/600
3/3 [==============================] - 0s 76ms/step - loss: 0.7006 - acc: 0.4948
Epoch 5/600
3/3 [==============================] - 0s 75ms/step - loss: 0.6863 - acc: 0.5469
Epoch 6/600
3/3 [==============================] - 0s 74ms/step - loss: 0.6963 - acc: 0.4740
Epoch 7/600
3/3 [==============================] - 0s 78ms/step - loss: 0.6924 - acc: 0.4896
Epoch 8/600
3/3 [==============================] - 0s 74ms/step - loss: 0.6926 - acc: 0.5677
Epoch 9/600
3/3 [==============================] - 0s 73ms/step - loss: 0.6892 - acc: 0.5208
Epoch 10/600
3/3 [==============================] - 0s 77ms/step - loss: 0.6803 - acc: 0.5625
Epoch 11/600
3/3 [==============================] - 0s 74ms/step - loss: 0.6782 -

In [0]:
tf.keras.models.save_model(model, "savedModel", overwrite=True, include_optimizer=True)